# Using btrack with napari


In [1]:
import os
import numpy as np
import btrack
import napari

In [2]:
objects = btrack.dataio.import_JSON('objects.json')

In [3]:
# with btrack.dataio.HDF5FileHandler('/home/quantumjot/Dropbox/Code/py3/arboretum/tracks.h5', 'r') as h:
#     segmentation = h._hdf['segmentation']['images'][:]
#     objects = h.objects

In [4]:
# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:

  # configure the tracker using a config file
  tracker.configure_from_file('../models/cell_config.json')

  # append the objects to be tracked
  tracker.append(objects)

  # set the volume (Z axis volume is set very large for 2D data)
  tracker.volume=((0,1200), (0,1600), (-1e5,1e5))

  # track them (in interactive mode)
  tracker.track_interactive(step_size=100)

  # generate hypotheses and run the global optimiser
  tracker.optimize()

  # get the tracks as a python list
  data, properties, graph = tracker.to_napari()

[INFO][2020/09/29 09:31:32 AM] Loaded btrack: /home/quantumjot/Dropbox/Code/py3/BayesianTracker/btrack/libs/libtracker.so
[INFO][2020/09/29 09:31:32 AM] btrack (v0.3.12) library imported
[INFO][2020/09/29 09:31:32 AM] Setting max XYZ search radius to: 100
[INFO][2020/09/29 09:31:32 AM] Starting BayesianTracker session
[INFO][2020/09/29 09:31:32 AM] Loading configuration file: ../models/cell_config.json
[INFO][2020/09/29 09:31:32 AM] Loading motion model: b'cell_motion'
[INFO][2020/09/29 09:31:32 AM] Objects are of type: <class 'list'>
[INFO][2020/09/29 09:31:32 AM] Set volume to ((0, 1200), (0, 1600), (-100000.0, 100000.0))
[INFO][2020/09/29 09:31:32 AM] Starting tracking... 
[INFO][2020/09/29 09:31:32 AM] Tracking objects in frames 0 to 99 (of 500)...
[INFO][2020/09/29 09:31:32 AM]  - Timing (Bayesian updates: 1.08ms, Linking: 0.20ms)
[INFO][2020/09/29 09:31:32 AM]  - Probabilities (Link: 1.00000, Lost: 0.48361)
[INFO][2020/09/29 09:31:32 AM]  - Stats (Active: 78, Lost: 191, Conflicts

In [5]:
vertices = data[:, 1:]

In [6]:
# use the tracks PR to visualize the track data
with napari.gui_qt():
    viewer = napari.Viewer()
#     viewer.add_labels(segmentation)
    viewer.add_points(vertices, size=1, name='points', opacity=0.3)
    viewer.add_tracks(data, properties=properties, graph=graph, name='tracks')